# Grid search for hyperparameter tunning mop rfe fulldataset


# Import and setup


In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output
import time
import itertools

from mop_model import MoPModel as MoP_raw, MoPConfig


# Trainning and evaluation functions

In [62]:
def train_and_evaluate_trial_mop(params, X_train, y_train, X_val, y_val):
    """
    Trains and evaluates a single trial of the MoP model with a given set of hyperparameters.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # --- Model Configuration ---
    config = MoPConfig(
        input_dim=X_train.shape[1],
        output_dim=2,
        intermediate_dim=params['intermediate_dim'],
        layers=params['layers']
    )
    model = MoP_raw(config).to(device)
    
    # --- Data Preparation ---
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.LongTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.LongTensor(y_val).to(device)

    # --- Training with Early Stopping ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    max_epochs = 75

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        # MoP expects a sequence dimension
        y_pred, usage_losses, entropy_loss = model(X_train_tensor.unsqueeze(1))
        y_pred = y_pred.squeeze(1)
        
        # MoP auxiliary losses can be added here for more robust training
        class_loss = criterion(y_pred, y_train_tensor)
        total_loss = class_loss # For simplicity, we only use class_loss for early stopping
        
        total_loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            y_val_pred, _, _ = model(X_val_tensor.unsqueeze(1))
            val_loss = criterion(y_val_pred.squeeze(1), y_val_tensor)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            break

    # --- Final Evaluation on Validation Set ---
    model.eval()
    with torch.no_grad():
        y_pred_tensor, _, _ = model(X_val_tensor.unsqueeze(1))
        y_pred_tensor = y_pred_tensor.squeeze(1)
        probas = nn.functional.softmax(y_pred_tensor, dim=1)
        _, predicted = torch.max(probas, 1)
        
        y_true = y_val_tensor.cpu().numpy()
        y_pred = predicted.cpu().numpy()
        y_score = probas[:, 1].cpu().numpy()
        
        report = classification_report(y_true, y_pred, output_dict=True, zero_division=0).get('1', {})

        return {
            'intermediate_dim': params['intermediate_dim'],
            'layers': str(params['layers']), # Convert list to string for display
            'val_roc_auc': roc_auc_score(y_true, y_score),
            'val_accuracy': accuracy_score(y_true, y_pred),
            'val_f1_score': report.get('f1-score', 0),
            'val_precision': report.get('precision', 0),
            'val_recall': report.get('recall', 0),
            'val_cohen_kappa': cohen_kappa_score(y_true, y_pred)
        }


# Data Loading and Hyperparameter definition

In [ ]:
# --- 1. Load and Prepare Data ---
try:
    print("🔹 Loading and preparing data...")
    df = pd.read_csv('input_mop_rfe_fulldataset.csv', low_memory=False)
    X = df.drop(columns=['Dementia Status'])
    y = df['Dementia Status']
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    print(f"Data loaded successfully. Train shape: {X_train.shape}")
except FileNotFoundError:
    print("\n⚠️ ERROR: 'input_data.csv' not found. Please run the feature extractor first.")

# --- 2. Define the Hyperparameter Grid for MoP ---
param_grid = {
    'intermediate_dim': [32, 64, 96, 128],
    'layers': [
        ["0,8,16", "0,8,16", "0,8,16"],      
        ["0,16,32", "0,16,32", "0,16,32"],    
        ["0,8,16", "0,16,32", "0,8,16"],      
        ["0,16,32", "0,8,16", "0,16,32"]       
    ]
}

# Create all possible combinations
valid_params = [
    {'intermediate_dim': p[0], 'layers': p[1]}
    for p in itertools.product(param_grid['intermediate_dim'], param_grid['layers'])
]

print(f"\n🔹 Starting Grid Search for MoP. Total combinations to test: {len(valid_params)}")


🔹 Loading and preparing data...
Data loaded successfully. Train shape: (1297, 27)

🔹 Starting Grid Search for MoP. Total combinations to test: 16


# Run grid search and final evaluation

In [64]:
# --- 3. Run the Grid Search for MoP ---
results = []
for i, params in enumerate(valid_params):
    print(f"\n--- Testing Combination {i+1}/{len(valid_params)} ---")
    print(f"Parameters: {params}")
    
    result = train_and_evaluate_trial_mop(params, X_train_scaled, y_train.values, X_val_scaled, y_val.values)
    results.append(result)
    
    # Live update of results
    clear_output(wait=True)
    results_df = pd.DataFrame(results).sort_values('val_roc_auc', ascending=False)
    print("✅ Intermediate Tuning Results for MoP (Validation Set):")
    display(results_df)

# --- 4. Final Evaluation on Test Set ---
print("\n\n" + "="*40 + "\n✅ FINAL MoP EVALUATION ON HELD-OUT TEST SET\n" + "="*40)

# Get the best parameters from the grid search
best_params = results_df.iloc[0].to_dict()
print("🏆 Best MoP Hyperparameters found (based on validation ROC-AUC):")
print(f"  - Intermediate Dimension: {int(best_params['intermediate_dim'])}")
print(f"  - Layer Structure: {best_params['layers']}")

# Combine training and validation data
X_train_val = np.concatenate((X_train_scaled, X_val_scaled), axis=0)
y_train_val = np.concatenate((y_train.values, y_val.values), axis=0)

print("\nRetraining the best MoP model on combined Train+Validation data...")

# Create the final model with the best parameters
final_config = MoPConfig(
    input_dim=X_train_val.shape[1],
    output_dim=2,
    intermediate_dim=int(best_params['intermediate_dim']),
    layers=eval(best_params['layers']) # Use eval to convert string back to list
)
final_model = MoP_raw(final_config)

# Train the final model and evaluate on the test set
final_results = train_and_evaluate_trial_mop(
    {'intermediate_dim': int(best_params['intermediate_dim']), 'layers': eval(best_params['layers'])},
    X_train_val, y_train_val, X_test_scaled, y_test.values
)

print("\n--- Final MoP Performance on Test Set ---")
print(f"  - Accuracy:    {final_results['val_accuracy']:.4f}")
print(f"  - ROC-AUC:     {final_results['val_roc_auc']:.4f}")
print(f"  - F1-Score:    {final_results['val_f1_score']:.4f}")
print(f"  - Precision:   {final_results['val_precision']:.4f}")
print(f"  - Recall:      {final_results['val_recall']:.4f}")
print(f"  - Cohen Kappa: {final_results['val_cohen_kappa']:.4f}")

✅ Intermediate Tuning Results for MoP (Validation Set):


,intermediate_dim,layers,val_roc_auc,val_accuracy,val_f1_score,val_precision,val_recall,val_cohen_kappa
9,96,"['0,16,32', '0,16,32', '0,16,32']",0.788554,0.733813,0.746575,0.721854,0.773050,0.466964
1,32,"['0,16,32', '0,16,32', '0,16,32']",0.788218,0.712230,0.727891,0.699346,0.758865,0.423625
12,128,"['0,8,16', '0,8,16', '0,8,16']",0.786587,0.726619,0.744966,0.707006,0.787234,0.452217
15,128,"['0,16,32', '0,8,16', '0,16,32']",0.784516,0.715827,0.734007,0.698718,0.773050,0.430653
8,96,"['0,8,16', '0,8,16', '0,8,16']",0.784309,0.726619,0.730496,0.730496,0.730496,0.453124
0,32,"['0,8,16', '0,8,16', '0,8,16']",0.783973,0.712230,0.731544,0.694268,0.773050,0.423386
10,96,"['0,8,16', '0,16,32', '0,8,16']",0.783403,0.723022,0.733564,0.716216,0.751773,0.445527
5,64,"['0,16,32', '0,16,32', '0,16,32']",0.782963,0.719424,0.738255,0.700637,0.780142,0.437801
6,64,"['0,8,16', '0,16,32', '0,8,16']",0.781747,0.719424,0.741722,0.695652,0.794326,0.437568
7,64,"['0,16,32', '0,8,16', '0,16,32']",0.781617,0.705036,0.721088,0.692810,0.751773,0.409216




✅ FINAL MoP EVALUATION ON HELD-OUT TEST SET
🏆 Best MoP Hyperparameters found (based on validation ROC-AUC):
  - Intermediate Dimension: 96
  - Layer Structure: ['0,16,32', '0,16,32', '0,16,32']

Retraining the best MoP model on combined Train+Validation data...

--- Final MoP Performance on Test Set ---
  - Accuracy:    0.7590
  - ROC-AUC:     0.8356
  - F1-Score:    0.7581
  - Precision:   0.7664
  - Recall:      0.7500
  - Cohen Kappa: 0.5180
